#Tool-calling Agent

This is an auto-generated notebook created by an AI Playground export.

This notebook uses [Mosaic AI Agent Framework](https://docs.databricks.com/generative-ai/agent-framework/build-genai-apps.html) to recreate your agent from the AI Playground. It  demonstrates how to develop, manually test, evaluate, log, and deploy a tool-calling agent in LangGraph.

The agent code implements [MLflow's ChatAgent](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent) interface, a Databricks-recommended open-source standard that simplifies authoring multi-turn conversational agents, and is fully compatible with Mosaic AI agent framework functionality.

 **_NOTE:_**  This notebook uses LangChain, but AI Agent Framework is compatible with any agent authoring framework, including LlamaIndex or pure Python agents written with the OpenAI SDK.

## Prerequisites

- Address all `TODO`s in this notebook.

In [0]:
%pip install -U -qqqq mlflow langchain langgraph==0.3.4 databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks] uv
%pip install -U langchain-nimble langchain-openai
%pip install -U -qqqq mlflow backoff databricks-openai databricks-agents uv
%pip install -U -qqqq mlflow databricks-langchain databricks-agents uv langgraph==0.3.4
%pip install -U -qqqq langchain_mcp_adapters
%pip install -U -qqqq mlflow langchain langgraph==0.3.4 pydantic unitycatalog-langchain[databricks] uv
%pip install -U -q openai mlflow databricks-agents
%restart_python
dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.5/352.5 kB 10.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packa

## Define the agent in code
Below we define our agent code in a single cell, enabling us to easily write it to a local Python file for subsequent logging and deployment using the `%%writefile` magic command.

For more examples of tools to add to your agent, see [docs](https://docs.databricks.com/generative-ai/agent-framework/agent-tool.html).

## Test the agent

Interact with the agent to test its output. Since this notebook called `mlflow.langchain.autolog()` you can view the trace for each step the agent takes.

Replace this placeholder input with an appropriate domain-specific example for your agent.

In [0]:
dbutils.library.restartPython()

In [0]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import AIMessage

# === STEP 1: Load your agent1 (tool-calling LangGraphChatAgent) ===
from agent1 import agent as agent1_graph  # already compiled in agent1.py

# === STEP 2: Load your agent2 (ReAct agent created using create_react_agent) ===
from agent2 import load_agent
agent2_react = await load_agent()
# from agent2 import agent as agent2_react  # agent2.py exports a create_react_agent

# === STEP 3: Define shared state ===
class AgentState(TypedDict):
    user_input: str
    prompt_query: Optional[str]
    final_output: Optional[str]

# === STEP 4: Wrap agent1 execution as a node ===
def run_agent1(state: AgentState) -> AgentState:
    print("[Agent 1] Running tool-calling agent to generate structured query...")
    response = agent1_graph.invoke({"messages": [{"role": "user", "content": state["user_input"]}]})
    extracted_message = response["messages"][-1]["content"]
    print("Based on your requirement: here's the summary and your intelligent assistant will start to generate recommendations for you!")
    print(extracted_message)
    return {"user_input": state["user_input"], "prompt_query": extracted_message}

# === STEP 5: Wrap agent2 execution as a node ===
async def run_agent2(state: AgentState) -> AgentState:
    print("Running a google map live search using Nimble to find recommendations based on user requirements:")
    response = await agent2_react.ainvoke({
        "messages": f"Find information about: {state['user_input']}. Provide a concise summary."
    })

    messages = response.get("messages", [])
    ai_responses = [msg for msg in messages if isinstance(msg, AIMessage)]

    if ai_responses:
        print("✅ Latest AI Message:\n")
        print(ai_responses[-1].content)
    else:
        print("⚠️ No AIMessage found.")


    return {
        "user_input": state["user_input"],
        # "prompt_query": state["prompt_query"],
        "final_output": ai_responses[-1].content
    }

# === STEP 6: Build LangGraph chain ===
builder = StateGraph(AgentState)
builder.add_node("ParseQueryWithAgent1", RunnableLambda(run_agent1))
builder.add_node("ExecuteSearchWithAgent2", RunnableLambda(run_agent2))

builder.set_entry_point("ParseQueryWithAgent1")
builder.add_edge("ParseQueryWithAgent1", "ExecuteSearchWithAgent2")
builder.add_edge("ExecuteSearchWithAgent2", END)

graph = builder.compile()


In [0]:
user_input = """
I'm 33 weeks pregnant, staying in downtown LA. I need to visit a clinic at 10am, and afterward, I’d love to have brunch nearby — maybe pregnancy-safe or Mexican food.
"""

result = await graph.ainvoke({"user_input": user_input})
print(result["final_output"])


[Agent 1] Running tool-calling agent to generate structured query...
Based on your requirement: here's the summary and your intelligent assistant will start to generate recommendations for you!
Based on your request, here's the outline for your needs:

**Recommendation Request:**
"Recommend me top 3 brunch restaurants within 3 miles of downtown LA, available after 11:00 AM, specializing in pregnancy-safe Mexican cuisine or with the highest pregnancy food safety ratings."

**Detailed Breakdown:**

1. **Time Requirement:** After 11:00 AM (allowing time for your 10:00 AM clinic appointment)

2. **Location Requirement:** Within 3 miles of downtown Los Angeles, CA

3. **Place Type:** Restaurant (brunch-focused)

4. **Other Requirements:** 
   - Pregnancy-safe food options (avoiding high-mercury fish, unpasteurized cheeses, undercooked meats)
   - Mexican cuisine preferred, or restaurants with strong pregnancy-friendly menu options
   - Establishments with high food safety standards suitable

[Trace(request_id=tr-761922e9361344128f82955cb2e5b7b4), Trace(request_id=tr-55ae4a03500543679803b539fc887bdc), Trace(request_id=tr-407cdf4d57bd4f32b9832e3f60c02bb1)]

To Be Continued

### Log the `agent` as an MLflow model
Determine Databricks resources to specify for automatic auth passthrough at deployment time
- **TODO**: If your Unity Catalog Function queries a [vector search index](https://docs.databricks.com/generative-ai/agent-framework/unstructured-retrieval-tools.html) or leverages [external functions](https://docs.databricks.com/generative-ai/agent-framework/external-connection-tools.html), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See [docs](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#specify-resources-for-automatic-authentication-passthrough) for more details.

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

## Evaluate the agent with [Agent Evaluation](https://docs.databricks.com/generative-ai/agent-evaluation/index.html)

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics.

To evaluate your tool calls, try adding [custom metrics](https://docs.databricks.com/generative-ai/agent-evaluation/custom-metrics.html#evaluating-tool-calls).

## Perform pre-deployment validation of the agent
Before registering and deploying the agent, we perform pre-deployment checks via the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See [documentation](https://docs.databricks.com/machine-learning/model-serving/model-serving-debug.html#validate-inputs) for details

## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

## Deploy the agent

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See [docs](https://docs.databricks.com/generative-ai/deploy-agent.html) for details